In [1]:
import pandas as pd
import geopandas as gpd
import sys
sys.path.append(r'C:\Users\mfpen\Documents\Repositorios\geoloc2\Detecciondeterrenos')
from codigos import obtener_curt

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\mfpen\AppData\Local\Temp\ipykernel_8132\2563911923.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedo

In [2]:
CURT=gpd.read_file(r"D:\Secretaria\Bases_shp_INEGI\CURT\024 Atlacomulco\ATLACOMULCO_PREDIOS.shp")
CURT

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-99.87493 19.79576, -99.87494 19.795..."
1,15,024,0000,0240109128000000,194737177109953100419,NaN,"POLYGON ((-99.88603 19.79363, -99.88612 19.793..."
2,15,024,0000,0240109116000000,194736032409953102406,NaN,"POLYGON ((-99.88603 19.79363, -99.88590 19.793..."
3,15,024,0000,0240142005000000,194824758809952102202,NaN,"POLYGON ((-99.86944 19.80691, -99.86948 19.806..."
4,15,024,0000,0240144837000000,194744232409952299340,NaN,"POLYGON ((-99.87493 19.79576, -99.87485 19.795..."
...,...,...,...,...,...,...,...
1919,15,024,0000,0240122320000000,194719536809952058052,NaN,"POLYGON ((-99.86814 19.78877, -99.86809 19.788..."
1920,15,024,0000,0240122321000000,194720252909952063672,NaN,"POLYGON ((-99.86820 19.78889, -99.86814 19.788..."
1921,15,024,0000,0240122328000000,194720252909952058206,NaN,"POLYGON ((-99.86827 19.78906, -99.86820 19.788..."
1922,15,024,0000,0240122345000000,194720239609952072994,NaN,"POLYGON ((-99.86859 19.78904, -99.86872 19.788..."


In [3]:
casas_filtro=gpd.read_file(r"H:\.shortcut-targets-by-id\19j98C-PJkrk3gkgU7V03dLiJ4Gn-hnNe\geoshaps\Atlacomulco\Rotacion_Atlacomulco\prueba_deteccion_3_rotacion.shp")

In [4]:
casas_filtro["ID_casas"]=casas_filtro.index
casas_filtro

,clase_dete,conf,area,status,geometry,ID_casas
0,establecimiento,0.70136,511.425330,correcto,"POLYGON ((-11128888.95573 2257450.58082, -1112...",0
1,casas,0.82655,182.508647,correcto,"POLYGON ((-11128894.36396 2257458.17207, -1112...",1
2,establecimiento,0.55637,322.899914,correcto,"POLYGON ((-11128702.18241 2256321.84985, -1112...",2
3,establecimiento,0.55807,465.296771,correcto,"POLYGON ((-11128709.79818 2256311.39002, -1112...",3
4,establecimiento,0.65544,187.522621,correcto,"POLYGON ((-11128695.31023 2256334.84175, -1112...",4
...,...,...,...,...,...,...
58498,establecimiento,0.38781,1073.993192,correcto,"POLYGON ((-11100399.98801 2243752.76801, -1110...",58498
58499,terreno_baldio,0.53138,361.006115,correcto,"POLYGON ((-11099977.25028 2245867.52183, -1109...",58499
58500,establecimiento,0.62800,210.586900,correcto,"POLYGON ((-11099543.48043 2242089.02879, -1109...",58500
58501,establecimiento,0.24212,195.544979,correcto,"POLYGON ((-11099434.89340 2243549.85882, -1109...",58501


In [5]:
# CURT.geometry.value_counts()
casas_filtro.geometry.value_counts()

POLYGON ((-11128888.956 2257450.581, -11128884...    1
POLYGON ((-11128498.924 2255981.554, -11128488...    1
POLYGON ((-11128547.188 2255927.350, -11128543...    1
POLYGON ((-11128571.602 2255995.851, -11128569...    1
POLYGON ((-11128702.182 2256321.850, -11128697...    1
                                                    ..
POLYGON ((-11100394.036 2243756.817, -11100369...    1
POLYGON ((-11100399.988 2243752.768, -11100383...    1
POLYGON ((-11099977.250 2245867.522, -11099959...    1
POLYGON ((-11099543.480 2242089.029, -11099532...    1
POLYGON ((-11099425.118 2242077.771, -11099392...    1
Name: geometry, Length: 58503, dtype: int64

In [9]:
#Se revisa el CRS de cada base para poder hacer el cruce entre ellas
print("crs CURT: ",CURT.crs)
print("crs casas: ",casas_filtro.crs)

crs CURT:  EPSG:3857
crs casas:  EPSG:3857


In [7]:
CURT=CURT.set_crs(6364,allow_override=True)
# CURT.geometry.head()
CURT=CURT.to_crs(3857)
CURT.geometry

0       POLYGON ((-11118026.626 2248851.074, -11118027...
1       POLYGON ((-11119261.940 2248599.490, -11119271...
2       POLYGON ((-11119261.940 2248599.490, -11119247...
3       POLYGON ((-11117414.936 2250171.302, -11117419...
4       POLYGON ((-11118026.626 2248851.074, -11118017...
                              ...                        
1919    POLYGON ((-11117270.073 2248025.083, -11117265...
1920    POLYGON ((-11117276.877 2248038.361, -11117270...
1921    POLYGON ((-11117284.711 2248059.038, -11117276...
1922    POLYGON ((-11117320.510 2248056.753, -11117335...
1923    POLYGON ((-11117291.290 2248079.719, -11117288...
Name: geometry, Length: 1924, dtype: geometry

In [8]:
casas_filtro=casas_filtro.set_crs(3857,allow_override=True)
# casas_filtro.crs
# # CURT.geometry.head()
# casas_filtro.to_crs(3857)
casas_filtro.geometry

0        POLYGON ((-11128888.956 2257450.581, -11128884...
1        POLYGON ((-11128894.364 2257458.172, -11128890...
2        POLYGON ((-11128702.182 2256321.850, -11128697...
3        POLYGON ((-11128709.798 2256311.390, -11128704...
4        POLYGON ((-11128695.310 2256334.842, -11128692...
                               ...                        
58498    POLYGON ((-11100399.988 2243752.768, -11100383...
58499    POLYGON ((-11099977.250 2245867.522, -11099959...
58500    POLYGON ((-11099543.480 2242089.029, -11099532...
58501    POLYGON ((-11099434.893 2243549.859, -11099430...
58502    POLYGON ((-11099425.118 2242077.771, -11099392...
Name: geometry, Length: 58503, dtype: geometry

In [10]:
casas_filtro=obtener_curt(casas_filtro,'geometry')
casas_filtro

C:\Users\mfpen\Documents\Repositorios\geoloc2\Detecciondeterrenos\codigos.py:347: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  data['centroid'] = data[geom_col].to_crs(4326).centroid


,clase_dete,conf,area,status,geometry,ID_casas,CURT_f
0,establecimiento,0.70136,511.425330,correcto,"POLYGON ((-11128888.956 2257450.581, -11128884...",0,19520650389958203663
1,casas,0.82655,182.508647,correcto,"POLYGON ((-11128894.364 2257458.172, -11128890...",1,19520678009958209589
2,establecimiento,0.55637,322.899914,correcto,"POLYGON ((-11128702.182 2256321.850, -11128697...",2,19513214269958144775
3,establecimiento,0.55807,465.296771,correcto,"POLYGON ((-11128709.798 2256311.390, -11128704...",3,19513178869958145855
4,establecimiento,0.65544,187.522621,correcto,"POLYGON ((-11128695.310 2256334.842, -11128692...",4,19513244849958143840
...,...,...,...,...,...,...,...
58498,establecimiento,0.38781,1073.993192,correcto,"POLYGON ((-11100399.988 2243752.768, -11100383...",58498,19450920559942586858
58499,terreno_baldio,0.53138,361.006115,correcto,"POLYGON ((-11099977.250 2245867.522, -11099959...",58499,19461399749942455637
58500,establecimiento,0.62800,210.586900,correcto,"POLYGON ((-11099543.480 2242089.029, -11099532...",58500,19441895799942316247
58501,establecimiento,0.24212,195.544979,correcto,"POLYGON ((-11099434.893 2243549.859, -11099430...",58501,19450350719942281620


In [11]:
CURT.shape, casas_filtro.shape

((1924, 7), (58503, 7))

In [12]:
cruce_curt=gpd.sjoin(CURT,casas_filtro)
cruce_curt

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",31445,casas,0.35628,561.565068,correcto,31445,19474431089952303057
4,15,024,0000,0240144837000000,194744232409952299340,NaN,"POLYGON ((-11118026.626 2248851.074, -11118017...",31445,casas,0.35628,561.565068,correcto,31445,19474431089952303057
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",32129,establecimiento,0.45533,361.006115,correcto,32129,19474441869952297796
4,15,024,0000,0240144837000000,194744232409952299340,NaN,"POLYGON ((-11118026.626 2248851.074, -11118017...",32129,establecimiento,0.45533,361.006115,correcto,32129,19474441869952297796
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",32130,establecimiento,0.45707,140.391267,correcto,32130,19474460699952299548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1897,15,024,0000,0240147019000000,194838722209951267616,NaN,"POLYGON ((-11116090.129 2250610.380, -11116091...",41159,casas,0.37332,140.391267,correcto,41159,19483871169951267208
1900,15,024,0000,0240144236000000,194827684709952076215,NaN,"POLYGON ((-11117282.590 2250268.896, -11117312...",36387,casas,0.31278,877.445419,correcto,36387,19482696059952067821
1900,15,024,0000,0240144236000000,194827684709952076215,NaN,"POLYGON ((-11117282.590 2250268.896, -11117312...",36389,establecimiento,0.35705,375.045242,correcto,36389,19482707239952066180
1906,15,024,0000,0240214105000000,194717131209950579254,NaN,"POLYGON ((-11115184.555 2247934.957, -11115187...",43389,casas,0.28564,342.955809,correcto,43389,19471689709950580613


In [14]:
cruce_curt['CURT_si']=1
cruce_curt.tail(3)

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si
1900,15,024,0000,0240144236000000,194827684709952076215,NaN,"POLYGON ((-11117282.590 2250268.896, -11117312...",36389,establecimiento,0.35705,375.045242,correcto,36389,19482707239952066180,1
1906,15,024,0000,0240214105000000,194717131209950579254,NaN,"POLYGON ((-11115184.555 2247934.957, -11115187...",43389,casas,0.28564,342.955809,correcto,43389,19471689709950580613,1
1906,15,024,0000,0240214105000000,194717131209950579254,NaN,"POLYGON ((-11115184.555 2247934.957, -11115187...",43399,casas,0.73559,210.586900,correcto,43399,19471722329950575366,1


In [15]:
cruce_curt.geometry.unique()

<GeometryArray>
[<POLYGON ((-11118026.626 2248851.074, -11118027.811 2248847.925, -11118039.2...>,
 <POLYGON ((-11118026.626 2248851.074, -11118017.083 2248848.289, -11118019.7...>,
 <POLYGON ((-11119261.94 2248599.49, -11119271.573 2248591.083, -11119282.706...>,
 <POLYGON ((-11119261.94 2248599.49, -11119247.839 2248579.669, -11119280.365...>,
 <POLYGON ((-11105996.084 2248177.528, -11105990.776 2248177.298, -11105975.9...>,
 <POLYGON ((-11106002.885 2248164.983, -11105976.342 2248163.832, -11105976.9...>,
 <POLYGON ((-11105995.653 2248190.349, -11105996.084 2248177.528, -11106002.4...>,
 <POLYGON ((-11105990.345 2248190.119, -11105990.776 2248177.298, -11105996.0...>,
 <POLYGON ((-11105975.911 2248176.653, -11105990.776 2248177.298, -11105990.3...>,
 <POLYGON ((-11106150.573 2247869.14, -11106159.393 2247869.155, -11106161.66...>,
 ...
 <POLYGON ((-11115075.635 2250634.278, -11115082.615 2250617.513, -11115112.4...>,
 <POLYGON ((-11115082.615 2250617.513, -11115075.635 2250634.278, 

In [16]:
no_cruzanxcurt=CURT[~(CURT.geometry.isin(cruce_curt['geometry']))]
no_cruzanxcurt

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry
3,15,024,0000,0240142005000000,194824758809952102202,NaN,"POLYGON ((-11117414.936 2250171.302, -11117419..."
5,15,024,0000,0240602413000000,195117681409955222284,NaN,"POLYGON ((-11123301.554 2255869.972, -11123312..."
10,15,024,0000,0241102980000000,194714662109946182809,NaN,"POLYGON ((-11106517.757 2247856.374, -11106558..."
11,15,024,0000,0241102978000000,194714818709946169924,NaN,"POLYGON ((-11106479.806 2247861.221, -11106517..."
96,15,024,0000,0241511115000000,194835629609949381160,NaN,"POLYGON ((-11112815.419 2250567.527, -11112691..."
...,...,...,...,...,...,...,...
1899,15,024,0000,0240147021000000,194840887209951271542,NaN,"POLYGON ((-11116081.256 2250713.011, -11116080..."
1901,15,024,0000,0240147022000000,194842015309951287433,NaN,"POLYGON ((-11116133.716 2250715.645, -11116144..."
1903,15,024,0000,0240147024000000,194842107009951266702,NaN,"POLYGON ((-11116069.050 2250713.645, -11116081..."
1904,15,024,0000,0240147006000000,194840706309951248676,NaN,"POLYGON ((-11115973.256 2250680.501, -11116000..."


In [17]:
no_cruzanxcurt['CURT_si']=0
no_cruzanxcurt

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,CURT_si
3,15,024,0000,0240142005000000,194824758809952102202,NaN,"POLYGON ((-11117414.936 2250171.302, -11117419...",0
5,15,024,0000,0240602413000000,195117681409955222284,NaN,"POLYGON ((-11123301.554 2255869.972, -11123312...",0
10,15,024,0000,0241102980000000,194714662109946182809,NaN,"POLYGON ((-11106517.757 2247856.374, -11106558...",0
11,15,024,0000,0241102978000000,194714818709946169924,NaN,"POLYGON ((-11106479.806 2247861.221, -11106517...",0
96,15,024,0000,0241511115000000,194835629609949381160,NaN,"POLYGON ((-11112815.419 2250567.527, -11112691...",0
...,...,...,...,...,...,...,...,...
1899,15,024,0000,0240147021000000,194840887209951271542,NaN,"POLYGON ((-11116081.256 2250713.011, -11116080...",0
1901,15,024,0000,0240147022000000,194842015309951287433,NaN,"POLYGON ((-11116133.716 2250715.645, -11116144...",0
1903,15,024,0000,0240147024000000,194842107009951266702,NaN,"POLYGON ((-11116069.050 2250713.645, -11116081...",0
1904,15,024,0000,0240147006000000,194840706309951248676,NaN,"POLYGON ((-11115973.256 2250680.501, -11116000...",0


In [18]:
cruce_curt.shape, no_cruzanxcurt.shape

((6839, 15), (183, 8))

In [48]:
cruce_final_curt=pd.concat([cruce_curt,no_cruzanxcurt])
cruce_final_curt

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si
0,15,098,0000,0980845763000000,192937370009913582692,NaN,"POLYGON Z ((-11046539.740 2213146.179 0.000, -...",213234.0,establecimiento,0.28247,26.115113,correcto,213234.0,19293757359913581141,1
75465,15,098,0000,0980845764000000,192937652809913582900,NaN,"POLYGON Z ((-11046540.908 2213163.135 0.000, -...",213234.0,establecimiento,0.28247,26.115113,correcto,213234.0,19293757359913581141,1
0,15,098,0000,0980845763000000,192937370009913582692,NaN,"POLYGON Z ((-11046539.740 2213146.179 0.000, -...",213237.0,establecimiento,0.30054,32.643892,correcto,213237.0,19293735819913579161,1
2540,15,098,0000,0980845712000000,192937149409913575671,NaN,"POLYGON Z ((-11046518.068 2213139.048 0.000, -...",213237.0,establecimiento,0.30054,32.643892,correcto,213237.0,19293735819913579161,1
27168,15,098,0000,0980845711000000,192937420709913575874,NaN,"POLYGON Z ((-11046518.695 2213147.904 0.000, -...",213237.0,establecimiento,0.30054,32.643892,correcto,213237.0,19293735819913579161,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125093,15,098,0000,0980164818000000,192816420309912381469,NaN,"POLYGON Z ((-11044064.540 2210510.074 0.000, -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
125101,15,098,0000,0980965211000000,192611308409916388320,NaN,"POLYGON Z ((-11051508.178 2206385.533 0.000, -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
125122,15,098,0000,0980519114000000,192719367909916541792,NaN,"POLYGON Z ((-11051982.561 2208628.131 0.000, -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
125144,15,098,0000,0980561918000000,192731636609916320700,NaN,"POLYGON Z ((-11051299.743 2209029.550 0.000, -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [49]:
#Para detectar duplicados x geometría
geo_dupli=cruce_final_curt.geometry.value_counts()[cruce_final_curt.geometry.value_counts()>1].index
cruce_final_curt.loc[cruce_final_curt.geometry.isin(geo_dupli),"dupGEO"]=1
cruce_final_curt.loc[cruce_final_curt.dupGEO != 1, "dupGEO"]=0
cruce_final_curt


,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si,dupGEO
0,15,098,0000,0980845763000000,192937370009913582692,NaN,"POLYGON Z ((-11046539.740 2213146.179 0.000, -...",213234.0,establecimiento,0.28247,26.115113,correcto,213234.0,19293757359913581141,1,1.0
75465,15,098,0000,0980845764000000,192937652809913582900,NaN,"POLYGON Z ((-11046540.908 2213163.135 0.000, -...",213234.0,establecimiento,0.28247,26.115113,correcto,213234.0,19293757359913581141,1,1.0
0,15,098,0000,0980845763000000,192937370009913582692,NaN,"POLYGON Z ((-11046539.740 2213146.179 0.000, -...",213237.0,establecimiento,0.30054,32.643892,correcto,213237.0,19293735819913579161,1,1.0
2540,15,098,0000,0980845712000000,192937149409913575671,NaN,"POLYGON Z ((-11046518.068 2213139.048 0.000, -...",213237.0,establecimiento,0.30054,32.643892,correcto,213237.0,19293735819913579161,1,1.0
27168,15,098,0000,0980845711000000,192937420709913575874,NaN,"POLYGON Z ((-11046518.695 2213147.904 0.000, -...",213237.0,establecimiento,0.30054,32.643892,correcto,213237.0,19293735819913579161,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125093,15,098,0000,0980164818000000,192816420309912381469,NaN,"POLYGON Z ((-11044064.540 2210510.074 0.000, -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
125101,15,098,0000,0980965211000000,192611308409916388320,NaN,"POLYGON Z ((-11051508.178 2206385.533 0.000, -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
125122,15,098,0000,0980519114000000,192719367909916541792,NaN,"POLYGON Z ((-11051982.561 2208628.131 0.000, -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
125144,15,098,0000,0980561918000000,192731636609916320700,NaN,"POLYGON Z ((-11051299.743 2209029.550 0.000, -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0


In [50]:
#Para detectar duplicados x curt
curt_dupli=cruce_final_curt.curt.value_counts()[cruce_final_curt.curt.value_counts()>1].index
cruce_final_curt.loc[cruce_final_curt.curt.isin(curt_dupli),"dupCURT"]=1
cruce_final_curt.loc[cruce_final_curt.dupCURT != 1, "dupCURT"]=0
cruce_final_curt

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT
0,15,098,0000,0980845763000000,192937370009913582692,NaN,"POLYGON Z ((-11046539.740 2213146.179 0.000, -...",213234.0,establecimiento,0.28247,26.115113,correcto,213234.0,19293757359913581141,1,1.0,1.0
75465,15,098,0000,0980845764000000,192937652809913582900,NaN,"POLYGON Z ((-11046540.908 2213163.135 0.000, -...",213234.0,establecimiento,0.28247,26.115113,correcto,213234.0,19293757359913581141,1,1.0,1.0
0,15,098,0000,0980845763000000,192937370009913582692,NaN,"POLYGON Z ((-11046539.740 2213146.179 0.000, -...",213237.0,establecimiento,0.30054,32.643892,correcto,213237.0,19293735819913579161,1,1.0,1.0
2540,15,098,0000,0980845712000000,192937149409913575671,NaN,"POLYGON Z ((-11046518.068 2213139.048 0.000, -...",213237.0,establecimiento,0.30054,32.643892,correcto,213237.0,19293735819913579161,1,1.0,1.0
27168,15,098,0000,0980845711000000,192937420709913575874,NaN,"POLYGON Z ((-11046518.695 2213147.904 0.000, -...",213237.0,establecimiento,0.30054,32.643892,correcto,213237.0,19293735819913579161,1,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125093,15,098,0000,0980164818000000,192816420309912381469,NaN,"POLYGON Z ((-11044064.540 2210510.074 0.000, -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0
125101,15,098,0000,0980965211000000,192611308409916388320,NaN,"POLYGON Z ((-11051508.178 2206385.533 0.000, -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0
125122,15,098,0000,0980519114000000,192719367909916541792,NaN,"POLYGON Z ((-11051982.561 2208628.131 0.000, -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0
125144,15,098,0000,0980561918000000,192731636609916320700,NaN,"POLYGON Z ((-11051299.743 2209029.550 0.000, -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0


In [51]:
casas_nocruzan = casas_filtro[~casas_filtro.ID_casas.isin(cruce_curt.ID_casas.unique())]
casas_nocruzan['YOLO_si'] = 0
casas_nocruzan.tail(2)

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,clase_dete,conf,area,status,geometry,ID_casas,CURT_f,YOLO_si
254508,establecimiento,0.50391,45.701448,correcto,"POLYGON ((-11043739.565 2210503.490, -11043738...",254508,19281674609912272021,0
254526,establecimiento,0.26807,32.643892,correcto,"POLYGON ((-11043742.924 2210399.845, -11043741...",254526,19281362269912273446,0


In [52]:
cruce_final_curt.shape, casas_nocruzan.shape

((440749, 17), (43887, 8))

In [53]:
cruce_final_curt.columns, casas_nocruzan.columns

(Index(['cve_ent', 'cve_mun', 'cve_loc', 'id_cat', 'curt', 'notas', 'geometry',
        'index_right', 'clase_dete', 'conf', 'area', 'status', 'ID_casas',
        'CURT_f', 'CURT_si', 'dupGEO', 'dupCURT'],
       dtype='object'),
 Index(['clase_dete', 'conf', 'area', 'status', 'geometry', 'ID_casas',
        'CURT_f', 'YOLO_si'],
       dtype='object'))

In [54]:
cruce_final =pd.concat([cruce_final_curt,casas_nocruzan])
cruce_final

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si
0,15,098,0000,0980845763000000,192937370009913582692,NaN,"POLYGON Z ((-11046539.740 2213146.179 0.000, -...",213234.0,establecimiento,0.28247,26.115113,correcto,213234.0,19293757359913581141,1.0,1.0,1.0,NaN
75465,15,098,0000,0980845764000000,192937652809913582900,NaN,"POLYGON Z ((-11046540.908 2213163.135 0.000, -...",213234.0,establecimiento,0.28247,26.115113,correcto,213234.0,19293757359913581141,1.0,1.0,1.0,NaN
0,15,098,0000,0980845763000000,192937370009913582692,NaN,"POLYGON Z ((-11046539.740 2213146.179 0.000, -...",213237.0,establecimiento,0.30054,32.643892,correcto,213237.0,19293735819913579161,1.0,1.0,1.0,NaN
2540,15,098,0000,0980845712000000,192937149409913575671,NaN,"POLYGON Z ((-11046518.068 2213139.048 0.000, -...",213237.0,establecimiento,0.30054,32.643892,correcto,213237.0,19293735819913579161,1.0,1.0,1.0,NaN
27168,15,098,0000,0980845711000000,192937420709913575874,NaN,"POLYGON Z ((-11046518.695 2213147.904 0.000, -...",213237.0,establecimiento,0.30054,32.643892,correcto,213237.0,19293735819913579161,1.0,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254444,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11043927.121 2210447.200, -11043926...",NaN,establecimiento,0.46680,31.918472,correcto,254444.0,19281498299912333834,NaN,NaN,NaN,0.0
254450,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11043911.459 2210478.341, -11043908...",NaN,establecimiento,0.56689,33.006602,correcto,254450.0,19281604059912328230,NaN,NaN,NaN,0.0
254495,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11043863.170 2210377.041, -11043856...",NaN,establecimiento,0.35352,32.643892,correcto,254495.0,19281278589912312100,NaN,NaN,NaN,0.0
254508,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11043739.565 2210503.490, -11043738...",NaN,establecimiento,0.50391,45.701448,correcto,254508.0,19281674609912272021,NaN,NaN,NaN,0.0


In [55]:
cruce_final.loc[cruce_final.YOLO_si.isna(),"YOLO_si"]=1

In [56]:
cruce_final.rename(columns={'index_right':'ID_curt'},inplace=True)

In [57]:
cruce_final.columns

Index(['cve_ent', 'cve_mun', 'cve_loc', 'id_cat', 'curt', 'notas', 'geometry',
       'ID_curt', 'clase_dete', 'conf', 'area', 'status', 'ID_casas', 'CURT_f',
       'CURT_si', 'dupGEO', 'dupCURT', 'YOLO_si'],
      dtype='object')

In [58]:
cruce_final.to_file(r"D:\Secretaria\cruces_bases\limpiezas_shp_base\Naucalpan/Naucalpan_casasxcurt.shp")

Cruce casas x industrias

In [26]:
path_industrial = r"H:\.shortcut-targets-by-id\19j98C-PJkrk3gkgU7V03dLiJ4Gn-hnNe\geoshaps\Ixtapan_Sal\industria\Ixtapan_industrias_filtro.shp"
industrias = gpd.read_file(path_industrial)

In [27]:
cruce_final.shape, industrias.shape

((43349, 19), (666, 11))

In [28]:
cruce_casasxind = gpd.sjoin(industrias,cruce_final)
cruce_casasxind

,puntos,zona,area_left,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,...,clase_dete,conf,area_right,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,establecimiento,0.81010,222.244914,correcto,8620.0,18513662109942335296,NaN,NaN,NaN,0.0
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,establecimiento,0.58994,502.816548,correcto,8614.0,18513711809942336682,NaN,NaN,NaN,0.0
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,establecimiento,0.20555,524.940476,correcto,8613.0,18513753159942340226,NaN,NaN,NaN,0.0
1,89,02,1080.604247,1137.882352,44.4497,25.7522,44.4497,25.7522,1.726054,0.949663,...,establecimiento,0.70573,896.019089,correcto,4469.0,18515161659943278317,NaN,NaN,NaN,0.0
1,89,02,1080.604247,1137.882352,44.4497,25.7522,44.4497,25.7522,1.726054,0.949663,...,establecimiento,0.65959,154.867497,correcto,4596.0,18515198549943272513,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,338,09,463.425800,626.140814,14.3218,43.2013,43.2013,14.3218,3.016471,0.740130,...,establecimiento,0.71699,422.365900,correcto,1062.0,18502579659944516443,1.0,1.0,1.0,1.0
665,264,09,480.625273,721.417894,17.8614,39.7185,39.7185,17.8614,2.223706,0.666223,...,establecimiento,0.67255,99.557677,correcto,709.0,18503076299945091265,NaN,NaN,NaN,0.0
665,264,09,480.625273,721.417894,17.8614,39.7185,39.7185,17.8614,2.223706,0.666223,...,establecimiento,0.82027,130.732302,correcto,710.0,18503109779945092268,NaN,NaN,NaN,0.0
665,264,09,480.625273,721.417894,17.8614,39.7185,39.7185,17.8614,2.223706,0.666223,...,establecimiento,0.45620,844.731801,correcto,656.0,18503127659945095547,NaN,NaN,NaN,0.0


In [29]:
cruce_casasxind['Ind_si']=1
cruce_casasxind

,puntos,zona,area_left,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,...,conf,area_right,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si,Ind_si
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,0.81010,222.244914,correcto,8620.0,18513662109942335296,NaN,NaN,NaN,0.0,1
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,0.58994,502.816548,correcto,8614.0,18513711809942336682,NaN,NaN,NaN,0.0,1
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,0.20555,524.940476,correcto,8613.0,18513753159942340226,NaN,NaN,NaN,0.0,1
1,89,02,1080.604247,1137.882352,44.4497,25.7522,44.4497,25.7522,1.726054,0.949663,...,0.70573,896.019089,correcto,4469.0,18515161659943278317,NaN,NaN,NaN,0.0,1
1,89,02,1080.604247,1137.882352,44.4497,25.7522,44.4497,25.7522,1.726054,0.949663,...,0.65959,154.867497,correcto,4596.0,18515198549943272513,NaN,NaN,NaN,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,338,09,463.425800,626.140814,14.3218,43.2013,43.2013,14.3218,3.016471,0.740130,...,0.71699,422.365900,correcto,1062.0,18502579659944516443,1.0,1.0,1.0,1.0,1
665,264,09,480.625273,721.417894,17.8614,39.7185,39.7185,17.8614,2.223706,0.666223,...,0.67255,99.557677,correcto,709.0,18503076299945091265,NaN,NaN,NaN,0.0,1
665,264,09,480.625273,721.417894,17.8614,39.7185,39.7185,17.8614,2.223706,0.666223,...,0.82027,130.732302,correcto,710.0,18503109779945092268,NaN,NaN,NaN,0.0,1
665,264,09,480.625273,721.417894,17.8614,39.7185,39.7185,17.8614,2.223706,0.666223,...,0.45620,844.731801,correcto,656.0,18503127659945095547,NaN,NaN,NaN,0.0,1


In [30]:
no_cruzanxInd=industrias[~(industrias.geometry.isin(cruce_casasxind['geometry']))]
no_cruzanxInd.tail(2)

,puntos,zona,area,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,geometry
616,788,09,1036.208248,2095.615786,56.6767,36.9749,56.6767,36.9749,1.532843,0.494465,"POLYGON ((-11098462.080 2128663.912, -11098456..."
617,1059,09,1082.846819,2641.839049,97.1913,27.1819,97.1913,27.1819,3.575589,0.409884,"POLYGON ((-11098510.879 2128684.712, -11098458..."


In [31]:
no_cruzanxInd['Ind_si']=0
no_cruzanxInd.tail(2)

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,puntos,zona,area,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,geometry,Ind_si
616,788,09,1036.208248,2095.615786,56.6767,36.9749,56.6767,36.9749,1.532843,0.494465,"POLYGON ((-11098462.080 2128663.912, -11098456...",0
617,1059,09,1082.846819,2641.839049,97.1913,27.1819,97.1913,27.1819,3.575589,0.409884,"POLYGON ((-11098510.879 2128684.712, -11098458...",0


In [32]:
cruce_casasxind.shape, no_cruzanxInd.shape

((3858, 31), (40, 12))

In [33]:
crucexInd=pd.concat([cruce_casasxind,no_cruzanxInd])
crucexInd

,puntos,zona,area_left,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,...,area_right,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si,Ind_si,area
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,222.244914,correcto,8620.0,18513662109942335296,NaN,NaN,NaN,0.0,1,NaN
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,502.816548,correcto,8614.0,18513711809942336682,NaN,NaN,NaN,0.0,1,NaN
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,524.940476,correcto,8613.0,18513753159942340226,NaN,NaN,NaN,0.0,1,NaN
1,89,02,1080.604247,1137.882352,44.4497,25.7522,44.4497,25.7522,1.726054,0.949663,...,896.019089,correcto,4469.0,18515161659943278317,NaN,NaN,NaN,0.0,1,NaN
1,89,02,1080.604247,1137.882352,44.4497,25.7522,44.4497,25.7522,1.726054,0.949663,...,154.867497,correcto,4596.0,18515198549943272513,NaN,NaN,NaN,0.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,2037,01,NaN,13932.717368,121.3612,114.5437,121.3612,114.5437,1.059519,0.538523,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,7503.087828
583,359,01,NaN,1465.064960,46.2086,31.6484,46.2086,31.6484,1.460061,0.517228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,757.772910
615,623,09,NaN,1639.389766,27.7846,58.9836,58.9836,27.7846,2.122888,0.427024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,700.058549
616,788,09,NaN,2095.615786,56.6767,36.9749,56.6767,36.9749,1.532843,0.494465,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1036.208248


In [34]:
nocruzan_casas =cruce_final[~cruce_final.ID_casas.isin(cruce_casasxind.ID_casas.unique())]
nocruzan_casas['Induxcasa'] = 0
nocruzan_casas.tail(2)

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,cve_ent,cve_mun,cve_loc,id_cat,proceso,curt,notas,geometry,ID_curt,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si,Induxcasa
29206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11088590.924 2136573.546, -11088580...",NaN,establecimiento,0.28596,464.602490,correcto,29206.0,18503777899936373374,NaN,NaN,NaN,0.0,0
29207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11088580.218 2136547.235, -11088561...",NaN,casas,0.55884,162.912562,correcto,29207.0,18503736079936371225,NaN,NaN,NaN,0.0,0


In [35]:
cruce_final_ind =pd.concat([crucexInd,nocruzan_casas])
cruce_final_ind

,puntos,zona,area_left,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,...,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si,Ind_si,area,Induxcasa
0,149.0,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,correcto,8620.0,18513662109942335296,NaN,NaN,NaN,0.0,1.0,NaN,NaN
0,149.0,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,correcto,8614.0,18513711809942336682,NaN,NaN,NaN,0.0,1.0,NaN,NaN
0,149.0,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,correcto,8613.0,18513753159942340226,NaN,NaN,NaN,0.0,1.0,NaN,NaN
1,89.0,02,1080.604247,1137.882352,44.4497,25.7522,44.4497,25.7522,1.726054,0.949663,...,correcto,4469.0,18515161659943278317,NaN,NaN,NaN,0.0,1.0,NaN,NaN
1,89.0,02,1080.604247,1137.882352,44.4497,25.7522,44.4497,25.7522,1.726054,0.949663,...,correcto,4596.0,18515198549943272513,NaN,NaN,NaN,0.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,correcto,29203.0,18495609089936406801,NaN,NaN,NaN,0.0,NaN,120.675972,0.0
29204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,correcto,29204.0,18503728779936376677,NaN,NaN,NaN,0.0,NaN,290.627965,0.0
29205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,correcto,29205.0,18503596579936391706,NaN,NaN,NaN,0.0,NaN,231.295612,0.0
29206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,correcto,29206.0,18503777899936373374,NaN,NaN,NaN,0.0,NaN,464.602490,0.0


In [36]:
cruce_final_ind.rename(columns={"index_right":"ID_ind"},inplace=True)
cruce_final_ind.columns

Index(['puntos', 'zona', 'area_left', 'rectangulo', 'Medida_lx', 'Medida_ly',
       'Max_xy', 'Min_xy', 'f_lado', 'f_area', 'geometry', 'ID_ind', 'cve_ent',
       'cve_mun', 'cve_loc', 'id_cat', 'proceso', 'curt', 'notas', 'ID_curt',
       'clase_dete', 'conf', 'area_right', 'status', 'ID_casas', 'CURT_f',
       'CURT_si', 'dupGEO', 'dupCURT', 'YOLO_si', 'Ind_si', 'area',
       'Induxcasa'],
      dtype='object')

In [37]:
cruce_final_ind.to_file(r"D:\Secretaria\cruces_bases\limpiezas_shp_base\Ixtapan\ultimas_versiones/Ixtapan_casas_Indxcurt.shp")